In [138]:
library(igraph)
library(ontologyIndex)
library(ontologySimilarity)
library(utils)
library(reshape2)
library(foreach)
library(doParallel)
library(data.table)

In [2]:
a=read.delim("../analysis/GENIE3.top10_target_ranks.ranked_regulator_target_interactions.with_communities.correlations_and_directionality.tsv.gz",sep="\t",stringsAsFactors=FALSE)
nrow(a)
a=a[a$community!="0",]
a=a[a$community %in% as.character(as.roman(1:11)),]
nrow(a)
head(a)


[1] 356457

[1] 215035

regulatoryGene,targetGene,full_cor,full_reg,our_cor,our_reg,weight,regulator_rank,target_rank,mutual_rank,regulator_community,target_community,community
Pp3c17_930.mRNA,Pp3s340_10.mRNA,0.1304851,+,1.874270e-15,+,0.06778052,1,1,1,I,I,I
Pp3c1_27430.pre_miRNA,Pp3c15_12100.mRNA,0.5589259,+,6.662628e-01,+,0.06724901,1,1,1,II,II,II
Pp3c17_17550.mRNA,Pp3c23_130.mRNA,0.8464697,+,8.916522e-01,+,0.06646932,1,1,1,II,II,II
Pp3c17_18920.pre_miRNA,Pp3s32_420.mRNA,1.0000000,+,1.000000e+00,+,0.06632740,1,1,1,III,III,III
Pp3c15_549.mRNA,Pp3c4_24040.mRNA,1.0000000,+,1.000000e+00,+,0.06400505,1,1,1,IV,IV,IV
Pp3c4_3072.pre_miRNA,Pp3c22_1240.mRNA,0.9517601,+,9.665704e-01,+,0.06353888,1,1,1,II,II,II


In [3]:
n=read.delim("../analysis/GENIE3.top10_target_ranks.communities_PLM.with_node_centralities.csv",sep=";",stringsAsFactors=FALSE)
n=n[n$community %in% as.character(as.roman(1:11)),]
head(n)

geneid,community,degree_centrality,betweenness_centrality,eigenvector_centrality,local_reaching_centrality,centrality_rank
Pp3c27_3080.mRNA,I,109,0,0.00000000,0.9977499,1
Pp3c11_23290.mRNA,I,552,3190,0.05359921,0.9971070,2
Pp3c7_2300.mRNA,I,505,7529,0.02518149,0.9971070,3
Pp3c17_23620.mRNA,I,490,1988,0.05419710,0.9971070,4
Pp3c16_13280.mRNA,I,485,9982,0.05634423,0.9971070,5
Pp3c15_11610.mRNA,I,461,3734,0.04049888,0.9971070,6


In [4]:
n$is_regulator=n$geneid %in% unique(a$regulatoryGene)

In [5]:
com=sapply(as.character(as.roman(1:11)),function(i) n[n$community==i, "geneid"])
str(com)

List of 11
 $ I   : chr [1:3112] "Pp3c27_3080.mRNA" "Pp3c11_23290.mRNA" "Pp3c7_2300.mRNA" "Pp3c17_23620.mRNA" ...
 $ II  : chr [1:4418] "Pp3c15_24790.mRNA" "Pp3c4_14601.pre_miRNA" "Pp3c22_13060.mRNA" "Pp3c9_13190.mRNA" ...
 $ III : chr [1:1705] "Pp3c13_10800.mRNA" "Pp3c22_11080.mRNA" "Pp3c16_9970.mRNA" "Pp3c8_250.mRNA" ...
 $ IV  : chr [1:2728] "Pp3c19_7560.mRNA" "Pp3c19_9880.mRNA" "Pp3c20_7710.mRNA" "Pp3c6_29020.mRNA" ...
 $ V   : chr [1:3435] "Pp3c22_2330.mRNA" "Pp3c5_12740.mRNA" "Pp3c8_6890.mRNA" "Pp3c7_25430.mRNA" ...
 $ VI  : chr [1:1849] "Pp3c16_20970.mRNA" "Pp3c1_32480.mRNA" "Pp3c10_13060.mRNA" "Pp3c4_25130.mRNA" ...
 $ VII : chr [1:3851] "Pp3c2_35433.mRNA" "Pp3c7_24670.mRNA" "Pp3c15_14600.mRNA" "Pp3c9_14240.mRNA" ...
 $ VIII: chr [1:3662] "Pp3c10_2910.mRNA" "Pp3c1_4600.mRNA" "Pp3c14_4440.mRNA" "Pp3c2_15500.mRNA" ...
 $ IX  : chr [1:6167] "Pp3c13_8290.mRNA" "Pp3c5_21130.mRNA" "Pp3c3_31770.mRNA" "Pp3c3_17380.mRNA" ...
 $ X   : chr [1:2403] "Pp3c1_15290.mRNA" "Pp3c16_15450.mRNA" "

In [6]:
G=graph_from_data_frame(a,directed = TRUE,vertices = n)
is_weighted(G)
list.edge.attributes(G)
list.vertex.attributes(G)

[1] TRUE

[1] "full_cor"            "full_reg"            "our_cor"            
 [4] "our_reg"             "weight"              "regulator_rank"     
 [7] "target_rank"         "mutual_rank"         "regulator_community"
[10] "target_community"    "community"

[1] "name"                      "community"                
[3] "degree_centrality"         "betweenness_centrality"   
[5] "eigenvector_centrality"    "local_reaching_centrality"
[7] "centrality_rank"           "is_regulator"

In [7]:
d=read.delim("analysis//significantly_enriched_terms.Benjamini-Hochberg.subnetwork_sets_NEAT.tsv",sep="\t",stringsAsFactors=FALSE)
d=subset(d,group=="subnetworks" & ontology %in% c("GO","PO") )#& p.adjusted<0.01 & fdr<0.01)
nrow(d)
table(d$partition)
head(d)

[1] 3722


               biological_process                cellular_component 
                             2167                               387 
               molecular_function           plant anatomical entity 
                              457                               548 
plant structure development stage 
                              163 

,group,set,ID,method,cutoff,ontology,Pop.total,Pop.term,Study.total,Study.term,⋯,name,community,A,nab,expected_nab,pvalue,conclusion,fdr,bonferroni,loci
5073,subnetworks,I,GO:0000160,Benjamini-Hochberg,0.1,GO,27502,317,2446,63,⋯,phosphorelay signal transduction system,I,I,467,56.9826,0.000000e+00,Overenrichment,0.000000e+00,0.000000e+00,"Pp3c10_20000.mRNA,Pp3c12_17730.mRNA,Pp3c5_810.mRNA,Pp3c10_20210.mRNA,Pp3c1_2880.mRNA,Pp3c20_13520.mRNA,Pp3c4_2680.mRNA,Pp3c20_3770.mRNA,Pp3c9_18950.mRNA,Pp3c11_6330.mRNA,Pp3c16_13280.mRNA,Pp3c20_17580.mRNA,Pp3c9_4870.mRNA,Pp3c1_2740.mRNA,Pp3c1_5010.mRNA,Pp3c16_18760.mRNA,Pp3c8_8150.mRNA,Pp3c1_32730.mRNA,Pp3c4_2530.mRNA,Pp3c11_10660.mRNA,Pp3c6_19350.mRNA,Pp3c9_4590.mRNA,Pp3c5_23340.mRNA,Pp3c25_12650.mRNA,Pp3c15_3620.mRNA,Pp3c2_1280.mRNA,Pp3c4_2590.mRNA,Pp3c11_23290.mRNA,Pp3c13_4270.mRNA,Pp3c6_28370.mRNA,Pp3c20_19070.mRNA,Pp3c3_31500.mRNA,Pp3c3_6830.mRNA,Pp3c25_7430.mRNA,Pp3c3_36750.mRNA,Pp3c6_16660.mRNA,Pp3c7_2300.mRNA,Pp3c16_300.mRNA,Pp3c1_2980.mRNA,Pp3c3_13510.mRNA,Pp3c6_28290.mRNA,Pp3c16_13260.mRNA,Pp3c7_7640.mRNA,Pp3c5_820.mRNA,Pp3c1_2770.mRNA,Pp3c5_9120.mRNA,Pp3c2_21330.mRNA,Pp3c9_25160.mRNA,Pp3c5_4120.mRNA,Pp3c3_23790.mRNA,Pp3c3_25560.mRNA,Pp3s423_10.mRNA,Pp3c5_3880.mRNA,Pp3c4_2660.mRNA,Pp3c27_1350.mRNA,Pp3c8_13750.mRNA,Pp3c18_4650.mRNA,Pp3c22_1800.mRNA,Pp3c14_17150.mRNA,Pp3c2_36690.mRNA,Pp3c19_6370.mRNA,Pp3c27_6030.mRNA,Pp3c1_32850.mRNA"
5074,subnetworks,I,GO:0000707,Benjamini-Hochberg,0.1,GO,27502,6,2446,3,⋯,meiotic DNA recombinase assembly,I,I,20,2.7135,1.707781e-14,Overenrichment,1.770757e-14,1.503018e-10,"Pp3c5_12400.mRNA,Pp3c23_10930.mRNA,Pp3c9_23740.mRNA"
5075,subnetworks,I,GO:0000723,Benjamini-Hochberg,0.1,GO,27502,102,2446,18,⋯,telomere maintenance,I,I,166,16.2807,4.489458e-156,Overenrichment,6.991987e-156,3.951172e-152,"Pp3c4_2680.mRNA,Pp3c20_3770.mRNA,Pp3c16_13280.mRNA,Pp3c1_5010.mRNA,Pp3c11_10660.mRNA,Pp3c6_19350.mRNA,Pp3c9_4590.mRNA,Pp3c15_3620.mRNA,Pp3c13_4270.mRNA,Pp3c2_20860.mRNA,Pp3c3_31500.mRNA,Pp3c3_6830.mRNA,Pp3c6_16660.mRNA,Pp3c6_28290.mRNA,Pp3c4_2660.mRNA,Pp3c2_36690.mRNA,Pp3c14_26730.mRNA,Pp3c27_6030.mRNA"
5076,subnetworks,I,GO:0000730,Benjamini-Hochberg,0.1,GO,27502,7,2446,3,⋯,DNA recombinase assembly,I,I,20,2.7135,1.707781e-14,Overenrichment,1.770757e-14,1.503018e-10,"Pp3c5_12400.mRNA,Pp3c23_10930.mRNA,Pp3c9_23740.mRNA"
5077,subnetworks,I,GO:0001505,Benjamini-Hochberg,0.1,GO,27502,29,2446,11,⋯,regulation of neurotransmitter levels,I,I,87,9.9493,2.550038e-70,Overenrichment,3.278727e-70,2.244288e-66,"Pp3c6_1770.mRNA,Pp3c3_5720.mRNA,Pp3c12_6540.mRNA,Pp3c13_2400.mRNA,Pp3c13_2420.mRNA,Pp3c5_10970.mRNA,Pp3c5_11640.mRNA,Pp3c12_6548.mRNA,Pp3c16_23390.mRNA,Pp3c13_3410.mRNA,Pp3c9_25180.mRNA"
5078,subnetworks,I,GO:0002020,Benjamini-Hochberg,0.1,GO,27502,57,2446,12,⋯,protease binding,I,I,82,10.8538,1.701386e-57,Overenrichment,2.088118e-57,1.497390e-53,"Pp3c3_25910.mRNA,Pp3c23_15640.mRNA,Pp3c5_3340.mRNA,Pp3c14_11830.mRNA,Pp3c2_28070.mRNA,Pp3c16_13851.mRNA,Pp3c20_23820.mRNA,Pp3c7_27100.mRNA,Pp3c20_11540.mRNA,Pp3c15_799.mRNA,Pp3c6_6330.mRNA,Pp3c1_25070.mRNA"


In [8]:
go=get_ontology("ontologies/go.obo", propagate_relationships=get_relation_names("ontologies/go.obo")[-c(4)])
po=get_ontology("ontologies/po.obo", propagate_relationships=get_relation_names("ontologies/po.obo")[-c(4)])

Warning message in (function (parents, id = names(parents), name = id, obsolete = setNames(nm = id, :
“Some parent terms not found: NCBITaxon:3398, NCBITaxon:33090”

In [139]:
goic=descendants_IC(go)
poic=descendants_IC(po)

In [9]:
go.no=go$id[! go$obsolete]
po.no=po$id[! po$obsolete]

In [10]:
gaf=read.delim("annotation//Physcomitrella_patens.GO.gaf2",sep="\t",stringsAsFactors=FALSE,header=FALSE,na="")
nrow(gaf)
gaf=gaf[gaf[,5] %in% go.no ,]
nrow(gaf)
head(gaf)

[1] 336337

[1] 331231

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17
cosmoss_PpV1.6TOV3.3,Pp3c10_10040.mRNA,Pp3c10_10040.mRNA,NA,GO:0005634,NA,ISS,cosmoss_V1.6:Pp1s58_283V6.1,C,NA,NA,gene_product,taxon:3218,20100908,cosmoss,NA,cosmoss_V3.3:Pp3c10_10040.mRNA
PGSB_pred2GO,Pp3c10_10040.mRNA,Pp3c10_10040.mRNA,NA,GO:0005829,NA,ISS,cosmoss_V3.3:Pp3c10_10040.mRNA,C,NA,NA,gene,taxon:3218,20190219,PGSB,NA,cosmoss_V3.3:Pp3c10_10040.mRNA
Pp_gene_atlas2GO,Pp3c10_10040.mRNA,Pp3c10_10040.mRNA,NA,GO:0032502,cosmoss_V3.3:Pp3c10_10040.mRNA,IEP,NA,P,NA,NA,gene,taxon:145481,20190217,PGSB,expressed_under(PECO:0007262),cosmoss_V3.3:Pp3c10_10040.mRNA
PGSB_pred2GO,Pp3c10_10090.mRNA,Pp3c10_10090.mRNA,NA,GO:0005576,NA,ISS,cosmoss_V3.3:Pp3c10_10090.mRNA,C,NA,NA,gene,taxon:3218,20190219,PGSB,NA,cosmoss_V3.3:Pp3c10_10090.mRNA
PGSB_pred2GO,Pp3c10_10090.mRNA,Pp3c10_10090.mRNA,NA,GO:0005618,NA,ISS,cosmoss_V3.3:Pp3c10_10090.mRNA,C,NA,NA,gene,taxon:3218,20190219,PGSB,NA,cosmoss_V3.3:Pp3c10_10090.mRNA
PGSB_pred2GO,Pp3c10_10090.mRNA,Pp3c10_10090.mRNA,NA,GO:0005634,NA,ISS,cosmoss_V3.3:Pp3c10_10090.mRNA,C,NA,NA,gene,taxon:3218,20190219,PGSB,NA,cosmoss_V3.3:Pp3c10_10090.mRNA


In [103]:
#registerDoSEQ()
cl=parallel::makeCluster(30, type="PSOCK")
doParallel::registerDoParallel(cl)

In [12]:
p=foreach(x=gaf[,5],.combine="c",.options.rsr=list(chunkSize=10000)) %dopar% {
    toupper(substr(go$name[go$ancestors[x][[1]][1]],1,1))
}

In [13]:
table(is.na(p))
table(p)


 FALSE 
331231 

p
     B      C      M 
171200  93713  66318 

In [14]:
p=ifelse(p=="B","P",ifelse(p=="M","F","C"))

In [15]:
gaf[,9]=p
table(p,gaf[,9])
table(is.na(gaf[,9]))

   
p        C      F      P
  C  93713      0      0
  F      0  66318      0
  P      0      0 171200


 FALSE 
331231 

In [16]:
paf=read.delim("annotation/Physcomitrella_patens.PO.gaf2",sep="\t",stringsAsFactors=FALSE,header=FALSE,na="")
nrow(paf)
paf=paf[paf[,5] %in% po.no & paf[,9] !="S" & ! is.na(paf[,9]),]
nrow(paf)
head(paf)

[1] 876665

[1] 656837

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17
cosmoss_PpV1.2TOV3.3,Pp3c10_10040.mRNA,Pp3c10_10040.mRNA,NA,PO:0000006,GB:PHYPADRAFT_163644|PMID:18762443|PMID:18079367,IEP,cosmoss_PpV1.2:Pp1s58_283V2.1,A,NA,Phypa_163644|PHYPADRAFT_163644,mRNA,taxon:145481,20110906,plantco.de|cosmoss.org,expressed_under(EO:0007265),cosmoss_V3.3:Pp3c10_10040.mRNA
Pp_gene_atlas2PO,Pp3c10_10040.mRNA,Pp3c10_10040.mRNA,NA,PO:0007134,cosmoss_V3.3:Pp3c10_10040.mRNA,IEP,NA,G,NA,NA,gene,taxon:145481,20190217,PGSB,expressed_under(PECO:0007265),cosmoss_V3.3:Pp3c10_10040.mRNA
cosmoss_PpV1.2TOV3.3,Pp3c10_10040.mRNA,Pp3c10_10040.mRNA,NA,PO:0025017,GB:PHYPADRAFT_163644|PMID:18762443|PMID:18079367,IEP,cosmoss_PpV1.2:Pp1s58_283V2.1,A,NA,Phypa_163644|PHYPADRAFT_163644,mRNA,taxon:145481,20110906,plantco.de|cosmoss.org,NA,cosmoss_V3.3:Pp3c10_10040.mRNA
Pp_gene_atlas2PO,Pp3c10_10040.mRNA,Pp3c10_10040.mRNA,NA,PO:0025075,cosmoss_V3.3:Pp3c10_10040.mRNA,IEP,NA,A,NA,NA,gene,taxon:145481,20190217,PGSB,expressed_under(PECO:0007262),cosmoss_V3.3:Pp3c10_10040.mRNA
Pp_gene_atlas2PO,Pp3c10_10040.mRNA,Pp3c10_10040.mRNA,NA,PO:0025094,cosmoss_V3.3:Pp3c10_10040.mRNA,IEP,NA,A,NA,NA,gene,taxon:145481,20190217,PGSB,expressed_under(PECO:0007265),cosmoss_V3.3:Pp3c10_10040.mRNA
Pp_gene_atlas2PO,Pp3c10_10040.mRNA,Pp3c10_10040.mRNA,NA,PO:0025345,cosmoss_V3.3:Pp3c10_10040.mRNA,IEP,NA,G,NA,NA,gene,taxon:145481,20190217,PGSB,expressed_under(PECO:0007262),cosmoss_V3.3:Pp3c10_10040.mRNA


In [17]:
table(paf[,9])
nrow(paf[is.na(paf[,9]),])


     A      G 
371523 285314 

[1] 0

In [18]:
maps=c("plant anatomical entity",
          "plant structure development stage",
          "molecular_function",
          "biological_process",
          "cellular_component")
names(maps)=c("A","G","M","P","C")
maps
spam=names(maps)
names(spam)=maps
spam

A                                   G 
          "plant anatomical entity" "plant structure development stage" 
                                  M                                   P 
               "molecular_function"                "biological_process" 
                                  C 
               "cellular_component"

plant anatomical entity plant structure development stage 
                              "A"                               "G" 
               molecular_function                biological_process 
                              "M"                               "P" 
               cellular_component 
                              "C"

In [19]:
A=data.table(rbind(unique(data.frame(ontology="GO",gaf[,c(9,2,5)])),data.frame(ontology="PO",unique(paf[,c(9,2,5)]))))
row.names(A)=NULL
names(A)[-1]=c("partition","geneid","ID")
A$partition=factor(maps[A$partition],levels=maps)
A$ontology=factor(A$ontology)
setkeyv(A,c("ID","geneid"))
head(A)
tail(A)

ontology,partition,geneid,ID
GO,biological_process,Pp3c23_17640.mRNA,GO:0000002
GO,biological_process,Pp3c8_22090.mRNA,GO:0000002
GO,biological_process,Pp3c11_16260.mRNA,GO:0000003
GO,biological_process,Pp3c11_20980.mRNA,GO:0000003
GO,biological_process,Pp3c11_21910.mRNA,GO:0000003
GO,biological_process,Pp3c14_10280.mRNA,GO:0000003


ontology,partition,geneid,ID
PO,plant anatomical entity,Pp3c19_14090.mRNA,PO:0030023
PO,plant anatomical entity,Pp3c1_32220.mRNA,PO:0030023
PO,plant anatomical entity,Pp3c21_1260.mRNA,PO:0030023
PO,plant anatomical entity,Pp3c22_12970.mRNA,PO:0030023
PO,plant anatomical entity,Pp3c22_17090.mRNA,PO:0030023
PO,plant anatomical entity,Pp3c2_5300.mRNA,PO:0030023


In [77]:
nrow(A)
nrow(na.omit(A))
A=na.omit(A)
nrow(A)

[1] 988068

[1] 921750

[1] 921750

In [141]:
O=list(GO=go,PO=po)
OI=list(GO=goic,PO=poic)

In [78]:
#D=subset(d, Study.term/Study.total>0.1)
DD=unique(d[,c("ID","ontology","partition","name")])
table(DD$partition)


               biological_process                cellular_component 
                             1463                               278 
               molecular_function           plant anatomical entity 
                              377                               236 
plant structure development stage 
                               72 

In [79]:
head(DD)

,ID,ontology,partition,name
5073,GO:0000160,GO,biological_process,phosphorelay signal transduction system
5074,GO:0000707,GO,biological_process,meiotic DNA recombinase assembly
5075,GO:0000723,GO,biological_process,telomere maintenance
5076,GO:0000730,GO,biological_process,DNA recombinase assembly
5077,GO:0001505,GO,biological_process,regulation of neurotransmitter levels
5078,GO:0002020,GO,molecular_function,protease binding


In [97]:
get_anc_mapped=function(id) {
    if (substr(id,1,2)=="GO") {
        anc=O[["GO"]]$ancestor[[id]]
    } 
    else {
        anc=O[["PO"]]$ancestor[[id]]
    }
    x=as.data.frame(A[id][,1:3])
    if (length(anc)>0) {
        return(do.call(rbind,lapply(anc, function(y) data.frame(x,ID=y))))
    } else {
        as.data.frame(cbind(x,ID=id))
    }
}

In [ ]:
AA=unique(foreach(id=unique(A$ID),.combine="rbind",.options.rsr=list(chunkSize=100),.packages=c("data.table"))  %dopar%  {
    get_anc_mapped(id)
})

In [111]:
length(unique(A$geneid))
length(unique(AA$geneid))

[1] 28999

[1] 28999

In [112]:
AAA=AA
AA=data.table(AA)
setkey(AA)
nrow(AA)
head(AA)

[1] 4431637

ontology,partition,geneid,ID
GO,biological_process,Pp3c10_10040.mRNA,GO:0008150
GO,biological_process,Pp3c10_10040.mRNA,GO:0032502
GO,biological_process,Pp3c10_10100.mRNA,GO:0008150
GO,biological_process,Pp3c10_10100.mRNA,GO:0050896
GO,biological_process,Pp3c10_10100.mRNA,GO:0006950
GO,biological_process,Pp3c10_10100.mRNA,GO:0042221


In [145]:
save(AAA,file="analysis/significantly_enriched_terms.ancestor_expanded_annotations.RData")

In [117]:
DD$ngenes=do.call(rbind,lapply(seq_len(nrow(DD)),function(i) nrow(AA[ID==DD[i,"ID"],"geneid"])))
head(DD)

,ID,ontology,partition,name,ngenes
5073,GO:0000160,GO,biological_process,phosphorelay signal transduction system,317
5074,GO:0000707,GO,biological_process,meiotic DNA recombinase assembly,6
5075,GO:0000723,GO,biological_process,telomere maintenance,108
5076,GO:0000730,GO,biological_process,DNA recombinase assembly,7
5077,GO:0001505,GO,biological_process,regulation of neurotransmitter levels,30
5078,GO:0002020,GO,molecular_function,protease binding,0


In [124]:
DD=subset(DD, ngenes>0)

In [125]:
analyseCommunities=function(i) {
    co=com[[i]]
    k=do.call(cbind,lapply(seq_len(nrow(DD)),function(i) nrow(AA[ID==DD[i,"ID"] & geneid %in% co,"geneid"])))
    k=matrix(k,nrow=1)
    rownames(k)=i
    colnames(k)=DD$ID
    k
}

In [126]:
outc=foreach(i=names(com),.combine="rbind",.options.rsr=list(chunkSize=60),.packages=c("igraph","data.table"))  %dopar%  {
    return(analyseCommunities(i))
}

In [142]:
DD$IC=sapply(seq_len(nrow(DD)), function(i) OI[[DD[i,"ontology"]]][DD[i,"ID"]])

In [143]:
write.table(outc,"analysis/significantly_enriched_terms.subnetworks.number_genes.csv",sep=";",quote=FALSE)
write.table(DD,"analysis//significantly_enriched_terms.Benjamini-Hochberg.subnetwork_sets_NEAT.total_number_of_genes_per_term.csv",sep=";",quote=FALSE)

In [130]:
analyseRegulators=function(i) {
    downstream=setdiff(V(G)[subcomponent(G,n[i,"geneid"],mode="out")]$name,n[i,"geneid"])
    k=do.call(cbind,lapply(seq_len(nrow(DD)),function(i) nrow(AA[ID==DD[i,"ID"] & geneid %in% downstream,"geneid"])))
    k=matrix(k,nrow=1)
    rownames(k)=n[i,"geneid"]
    colnames(k)=DD$ID
    k
}

In [35]:
nrow(unique(AA["GO:0002682","geneid"]))

[1] 3671

In [ ]:
out=foreach(i=which(n$is_regulator),.combine="rbind",.options.rsr=list(chunkSize=60),.packages=c("igraph","data.table"))  %dopar%  {
    return(analyseRegulators(i))
}

In [133]:
head(out)

,GO:0000160,GO:0000707,GO:0000723,GO:0000730,GO:0001505,GO:0002679,GO:0002682,GO:0003032,GO:0003677,GO:0003700,⋯,GO:0051248,GO:0051348,GO:0060359,GO:0070417,GO:0071497,GO:0072337,GO:0072662,GO:0072663,PO:0005018,PO:0008014
Pp3c27_3080.mRNA,63,3,18,3,11,27,84,3,0,2,⋯,11,8,35,7,4,3,6,6,0,0
Pp3c11_23290.mRNA,62,3,18,3,11,27,84,3,0,2,⋯,11,8,35,7,4,3,6,6,0,0
Pp3c7_2300.mRNA,62,3,18,3,11,27,84,3,0,2,⋯,11,8,35,7,4,3,6,6,0,0
Pp3c17_23620.mRNA,63,3,18,3,11,27,84,3,0,2,⋯,11,8,35,7,4,3,6,6,0,0
Pp3c16_13280.mRNA,62,3,17,3,11,26,84,3,0,2,⋯,11,8,35,7,4,3,6,6,0,0
Pp3c15_11610.mRNA,63,3,18,3,11,27,84,3,0,2,⋯,11,8,35,7,4,3,6,6,0,0


In [134]:
dim(out)
dim(DD)
dim(n)

[1] 1667 2084

[1] 2084    5

[1] 35639     8

In [135]:
stopCluster(cl)

In [136]:
write.table(out,"analysis/significantly_enriched_terms.subnetwork_sets.regulators.number_of_downstream_targets.csv",sep=";",quote=FALSE)